We will store all our data and Tables in database using SQLITE3

In [2]:
import pandas as pd
import numpy as np
import pickle
import joblib
import os

import datetime

import sqlite3
import json

In [3]:
# Loading all out tables
df = pd.read_parquet('../Data/data_with_features.parquet')
customer = pd.read_pickle('../Data/customer_history.pkl')
items = pd.read_pickle('../Data/item_summary.pkl')
baskets = pd.read_pickle('../Data/baskets.pkl')
itemsets = joblib.load('../Models/itemsets.joblib')
rules = pd.read_csv('../Data/rules.csv')
vectorizer = joblib.load('../Models/vectorizer.joblib')

Today = pd.Timestamp(df['Purchase Date'].max()) + pd.DateOffset(days=1)
Time_period = (df['Purchase Date'].max() - df['Purchase Date'].min()).days

In [4]:
# Normalize column names for DB consistency
def normalize_cols(df):
    df.columns = [c.replace(" ", "_") for c in df.columns]
    return df

df = normalize_cols(df)
customer = normalize_cols(customer)
items = normalize_cols(items)
baskets = normalize_cols(baskets)

In [5]:
df.columns

Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer_ID', 'Country', 'Purchase_Date', 'Purchase_Time',
       'Amount', 'Hour', 'PartOfDay', 'DayOfWeek', 'WeekOfYear', 'Year',
       'Clean_Description', 'description_length'],
      dtype='object')

In [6]:
path = '../Data/Database/Retail.db'


def get_connection(path):
    conn = sqlite3.connect(path)
    return conn

def _to_serializable(obj):
    """
    Convert types not JSON-serilazable into serializable types
    - pandas.Timestamp -> ISO string
    - numpy types -> Python scalar
    - lists/dicts -> recusively convert to serializable types
    """
    # pandas timestamp
    if isinstance(obj, (pd.Timestamp, datetime.datetime)):
        return obj.isoformat()
    # numpy scalars
    if isinstance(obj, (np.integer, np.int64, np.int32)):
        return int(obj)
    if isinstance(obj, (np.floating, np.float64, np.float32)):
        return float(obj)
    if isinstance(obj, (np.bool_, )):
        return bool(obj)
    # lists/tuples -> map recusively
    if isinstance(obj, (list, tuple)):
        return [_to_serializable(o) for o in obj]
    # dicts
    if isinstance(obj, dict):
        return {k: _to_serializable(v) for k, v in obj.items()}
    
    # default fallthrough
    return obj

def to_json(obj):
    return json.dumps(_to_serializable(obj), default=str)

def fetch_df(query, params=()):
    """Run a SELECT query and return results as a DataFrame."""
    conn = get_connection(path=path)
    df = pd.read_sql_query(query, conn, params=params)
    conn.close()
    return df

Step-1: Creating all Tables

In [32]:
# -- Customer Data --
def Create_Customers(Cursor):
    Cursor.execute("DROP TABLE IF EXISTS Customers")
    
    Cursor.execute("""
        CREATE TABLE Customers(
            Customer_ID INTEGER PRIMARY KEY,
            StockCode TEXT,
            Purchase_count TEXT,
            Purchase_quantity TEXT,
            Last_purchase_date TEXT
        );
    """)

# -- Items Data --
def Create_Items(Cursor):
    Cursor.execute("DROP TABLE IF EXISTS Items")
    
    Cursor.execute("""
        CREATE TABLE Items(
            StockCode TEXT PRIMARY KEY,
            Description TEXT,
            Current_Price REAL,
            Num_orders INTEGER,
            Total_quantity INTEGER,
            Total_sales REAL,
            Num_customers INTEGER,
            Last_sale TEXT,
            Frequency REAL,
            Quantity_per_order REAL,
            Quantity_per_customer REAL,
            Sales_per_customer REAL
        );
    """)

# -- Previous Transactions -- 
def Create_Transactions(Cursor):
    Cursor.execute("DROP TABLE IF EXISTS Transactions")
    # Dropping -> InvoiceDate, 
            # Defining table columns
    Cursor.execute("""
        CREATE TABLE Transactions(
            Invoice INTEGER,
            StockCode TEXT,
            Description TEXT,
            Quantity INTEGER,
            Price REAL,
            Customer_ID INTEGER,
            Country TEXT,
            Purchase_Date TEXT,
            Purchase_Time TEXT,
            Clean_Description TEXT,
            PRIMARY KEY (Invoice, StockCode),
            FOREIGN KEY (StockCode) references Items(StockCode),
            FOREIGN KEY (Customer_ID) references Customers(Customer_ID)
        );
    """)

# -- Baskets Tables --
def Create_Baskets(Cursor):
    Cursor.execute("DROP TABLE IF EXISTS Baskets")
    
    Cursor.execute("""
        CREATE TABLE Baskets(
            Invoice INTEGER PRIMARY KEY,
            Customer_ID INTEGER,
            Purchase_Time TEXT,
            Purchase_Date TEXT,
            StockCode TEXT,
            Description TEXT,
            Quantity TEXT,
            Price TEXT,
            Total_amount REAL,
            Num_products INTEGER,
            FOREIGN KEY (Customer_ID) references Customers(Customer_ID)
        );
    """)

> We need to serialize lists before adding them

This dataset's purchase history is old. So, to use it in our model, we need to make up current date and time.

In [33]:
# Inserting data row-by-row into the database
def Insert_customers_from_df(data):
    conn = get_connection(path)
    cursor = conn.cursor()
    
    for _, row in data.iterrows():
        try:
            cursor.execute("""
            INSERT or REPLACE INTO Customers (Customer_ID, StockCode, Purchase_count, Purchase_quantity, Last_purchase_date)
            VALUES (?, ?, ?, ?, ?)
            """, (int(row['Customer_ID']), to_json(row['StockCode']), to_json(row['Purchase_count']),
                  to_json(row['Purchase_quantity']), to_json(row['Last_purchase_date']))
            )
        except Exception as e:
            print(f"Failed inserting into customers with exception: {e}")
    
    conn.commit()
    conn.close()
    
def Insert_items_from_df(data):
    conn = get_connection(path)
    cursor = conn.cursor()
    
    for _, row in data.iterrows():
        try:
            cursor.execute("""
            INSERT or REPLACE INTO Items
            (StockCode, Description, Current_price, Num_orders, Total_quantity, Total_sales, 
             Num_customers, Last_sale, Frequency, Quantity_per_order, Quantity_per_customer, Sales_per_customer)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """, (row['StockCode'], row['Description'], row['Current_Price'], row['Num_orders'], row['Total_quantity'], row['Total_sales'], 
            row['Num_customers'], to_json(row['Last_sale']), row['Frequency'], row['Quantity_per_order'], row['Quantity_per_customer'], row['Sales_per_customer'])    )
        except Exception as e:
            print(f"Failed inserting into items with exception: {e}")
    
    conn.commit()
    conn.close()
    
def Insert_transactions_from_df(data):
    conn = get_connection(path)
    cursor = conn.cursor()
    
    for _, row in data.iterrows():
        try:
            cursor.execute("""
            INSERT or REPLACE INTO Transactions
            (Invoice, StockCode, Description, Quantity, Price, Customer_ID, Country,
                Purchase_Date, Purchase_Time, Clean_Description)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """, (row['Invoice'], row['StockCode'], row['Description'], row['Quantity'], row['Price'], row['Customer_ID'], row['Country'],
                   to_json(row['Purchase_Date']), to_json(row['Purchase_Time']), row['Clean_Description'])    )
        except Exception as e:
            print(f"Failed inserting into transactions with exception: {e}")
        
    conn.commit()
    conn.close()
    
def Insert_baskets_from_df(data):
    conn = get_connection(path)
    cursor = conn.cursor()
    
    for _, row in data.iterrows():
        try:
            cursor.execute("""
            INSERT or REPLACE INTO Baskets
            (Invoice, Customer_ID, Purchase_Time, Purchase_Date, StockCode, Description, 
            Quantity, Price, Total_amount, Num_products)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """, (row['Invoice'], row['Customer_ID'], to_json(row['Purchase_Time']), to_json(row['Purchase_Date']), to_json(row['StockCode']), to_json(row['Description']),
                to_json(row['Quantity']), to_json(row['Price']), float(row['Total_amount']), int(row['Num_products']))    )
        except Exception as e:
            print(f"Failed inserting into baskets with exception: {e}")
    
    conn.commit()
    conn.close()
    


In [34]:
# ------------------- Create Tables and Insert -------------------
conn = get_connection(path)
cur = conn.cursor()
Create_Customers(cur)
Create_Items(cur)
Create_Transactions(cur)
Create_Baskets(cur)
conn.commit()
conn.close()

# Populate DB
Insert_customers_from_df(customer)
Insert_items_from_df(items)
Insert_transactions_from_df(df)
Insert_baskets_from_df(baskets)

print("✅ All data successfully inserted into SQLite database at:", path)

✅ All data successfully inserted into SQLite database at: ../Data/Database/Retail.db


In [35]:
# def update_basket_from_transactions(Data):
#     try:
#         conn = get_connection(path)
#         cursor = conn.cursor()

#         invoice = int(Data['Invoice'])
#         customer_id = int(Data['Customer_ID'])
#         purchase_time = to_json(Data['Purchase_Time'])
#         purchase_date = to_json(Data['Purchase_Date'])
#         stockcode = to_json(Data['StockCode'])
#         description = to_json(Data['Description'])
#         quantity = to_json(Data['Quantity'])
#         price = to_json(Data['Price'])
#         total_amount = float(Data['Total_amount'])
#         num_products = int(Data['Num_products'])

#         # Check if basket already exists
#         cursor.execute("Select * from Baskets where Invoice = ?", (invoice,))
#         row = cursor.fetchone() # will only get single rows

#         if row is None:
#             # Create new basket
#             cursor.execute("""
#                 INSERT INTO Baskets 
#                 (Invoice, Customer_ID, Purchase_Time, Purchase_Date, StockCode, Description, Quantity, Price, Total_amount, Num_products)
#                 VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
#                 """, (invoice, customer_id, purchase_time, purchase_date, stockcode, description, quantity, price, total_amount, num_products))
#         else:
#             print(row)

#             # Update existing basket
#             cursor.execute("""
#                 UPDATE Baskets
#                 SET Customer_ID = ?, Purchase_Time = ?, Purchase_Date = ?, StockCode = ?, Description = ?, Quantity = ?, Price = ?, Total_amount = ?, Num_products = ?
#                 WHERE Invoice = ?
#                 """, (customer_id, purchase_time, purchase_date, stockcode, description, quantity, price, total_amount, num_products, invoice))

#         conn.commit()
#         conn.close()
    
#     except sqlite3.OperationalError as e:
#         print(f"Optional error: {e}")
#     except sqlite3.IntegrityError as e:
#         print(f"Integrity error: {e}")
#     except Exception as e:
#         print("Error occurred:", e)

Updating Baskets, Items, and Customers table one by one on new transaction is complex and computationally expensive.

Its better to just recreate a new item, customer and basket table using pandas and then save them to the database.

> Only use them to update items and customer table for new basket -> One basket at a time

In [36]:
# Conn = get_connection(path)
# pointer = Conn.cursor()

# Create_baskets(pointer)

# def Process_new_transaction(new_transaction, connection=Conn):
#     """ Inserts new transactions into Transactions table, recomputes Items, Customer, and Baskets tables
#         and update them in the database
        
#     Args:
#         new_transaction (pd.DataFrame): DataFrame containing the new transaction
#     """
#     cursor = connection.cursor()
    
#     # 1) Insert new transaction into Transactions table
#     Insert_transactions_from_df(new_transaction)
    
#     # 2) Fetch full transactions table from DB
#     transactions = pd.read_sql_query("SELECT * FROM Transactions", connection)
    
#     # # Convert serialized JSON columns back if needed
#     # (like StockCode, Quantity, Description lists)
#     for col in ['StockCode', 'Quantity', 'Description', 'Purchase_Date', 'Purchase_Time']:
#         if col in transactions.columns:
#             transactions[col] = transactions[col].apply(lambda x: json.loads(x) if x else x)
    
#     # 3) REcompute Item summary, Customer history, Baskets
#     items = compute_item_summary(transactions)
#     customer = compute_customer_summary(transactions)
#     baskets = compute_baskets(transactions)
    
#     # 4) Update tables in DB
#     Insert_items_from_df(items)
#     Insert_customers_from_df(customer)
#     Insert_baskets_from_df(baskets)
    

In [10]:
conn = get_connection(path)
display(pd.read_sql_query("SELECT * FROM Baskets",conn))
conn.close()

,Invoice,Customer_ID,Purchase_Time,Purchase_Date,StockCode,Description,Quantity,Price,Total_amount,Num_products
0,489434,13085,"""1900-01-01T07:45:00""","""2009-12-01T00:00:00""","[""21232"", ""21523"", ""21871""]","[""strawberry|ceramic|trinket|box"", ""fancy|font...","[24, 10, 24]","[1.25, 5.95, 1.25]",119.50,3
1,489435,13085,"""1900-01-01T07:46:00""","""2009-12-01T00:00:00""","[""22195""]","[""heart|measuring|spoons|large""]",[24],[1.65],39.60,1
2,489436,13078,"""1900-01-01T09:06:00""","""2009-12-01T00:00:00""","[""84879"", ""82582"", ""22296"", ""22111"", ""21755"", ...","[""assorted|colour|bird|ornament"", ""area|patrol...","[16, 12, 12, 24, 18, 3, 12]","[1.69, 2.1, 1.65, 4.25, 5.45, 5.95, 2.1]",315.19,7
3,489437,15362,"""1900-01-01T09:08:00""","""2009-12-01T00:00:00""","[""22112"", ""84970S"", ""22111"", ""20971"", ""21912""]","[""chocolate|hot|water|bottle"", ""hanging|heart|...","[3, 12, 3, 12, 4]","[4.95, 0.85, 4.95, 1.25, 3.75]",69.90,5
4,489438,18102,"""1900-01-01T09:24:00""","""2009-12-01T00:00:00""","[""20711""]","[""jumbo|bag|toys""]",[60],[1.3],78.00,1
...,...,...,...,...,...,...,...,...,...,...
31749,581579,17581,"""1900-01-01T12:19:00""","""2011-12-09T00:00:00""","[""85099C"", ""23199"", ""23201"", ""21929"", ""21931"",...","[""jumbo|bag|baroque|black|white"", ""jumbo|bag|a...","[10, 30, 20, 10, 10, 6, 12, 10, 10, 24, 12, 24...","[1.79, 1.79, 1.79, 1.79, 1.79, 2.95, 2.95, 1.7...",363.76,14
31750,581580,12748,"""1900-01-01T12:20:00""","""2011-12-09T00:00:00""","[""79321"", ""85049E"", ""20974"", ""21790"", ""22698""]","[""chilli|lights"", ""scandinavian|reds|ribbons"",...","[2, 2, 1, 1, 1]","[5.75, 1.25, 0.65, 0.85, 2.95]",18.45,5
31751,581583,13777,"""1900-01-01T12:23:00""","""2011-12-09T00:00:00""","[""20725""]","[""lunch|bag|red|retrospot""]",[40],[1.45],58.00,1
31752,581585,15804,"""1900-01-01T12:31:00""","""2011-12-09T00:00:00""","[""84946"", ""84879"", ""84692"", ""23084"", ""22727"", ...","[""antique|silver|t-light|glass"", ""assorted|col...","[12, 16, 25, 12, 4, 12, 4, 8]","[1.25, 1.69, 0.42, 2.08, 3.75, 1.95, 4.25, 3.75]",162.90,8


In [37]:
# ------------------- Quick verification -------------------
conn = get_connection(path)
for table in ['Customers', 'Items', 'Transactions', 'Baskets']:
    count = conn.execute(f"SELECT COUNT(*) FROM {table}").fetchone()[0]
    print(f"{table} → {count} rows")
conn.close()

Customers → 5332 rows
Items → 4861 rows
Transactions → 307722 rows
Baskets → 31754 rows


In [38]:
# import numpy as np
# import pickle

# Final_Model_weights= {'alpha': np.float32(2.1774216), 'beta': np.float32(3.9744666), 'gamma': np.float32(1.5444027), 'delta': np.float32(-0.4433973), 'eta': np.float32(0.53207666), 'epsilon': np.float32(0.35471776), 'd_effect': np.float32(1.9278386)}
# Validation_metrics = {'Precision@N': np.float64(0.007244147157190636), 'Recall@N': np.float64(0.7244147157190636), 'F1@N': np.float64(0.014344845713794867), 'HitRate': 0.7244147157190636, 'MRR@N': np.float64(0.09932481255115745), 'NDCG@N': np.float64(0.21643825650416038), 'MAP@N': np.float64(0.09932481255115745), 'Correct_predictions': 2166, 'Total_baskets': 2990}


# pickle.dump(Final_Model_weights, open("../experiments/Final_model_weights.pkl", "wb"))
# pickle.dump(Validation_metrics, open("../experiments/Final_model_metrics.pkl", "wb"))


In [39]:
# wt = pickle.load(open("../experiments/Final_model_weights.pkl", "rb"))
# print(wt)

# metrics = pickle.load(open("../experiments/Final_model_metrics.pkl", "rb"))
# print(metrics)